In [ ]:
import random
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
words = open("../names.txt").read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = dict([(c, i) for i, c in enumerate(['.']+chars)])
itos = dict([(i, c) for c, i in stoi.items()])
vocab_size = len(stoi.keys())
block_size = 3 # Context length

In [ ]:
# Making X and Y
def build_dataset(words):
    X, Y = [], []
    
    for w in words:
        context = [0] * 3 # Starting with 3 dots. Default that leads to the first Y token prediction.
        
        for ch in w + ".":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print("".join([itos[i] for i in context]) + " --> " + ch)
            context = context[1:] + [ix]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

random.seed = 42
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

# Train, dev, test
Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xtest, Ytest = build_dataset(words[n2:])

In [ ]:
print(Xtr.shape, Ytr.shape, Xdev.shape, Ydev.shape, Xtest.shape, Ytest.shape)

In [ ]:
# Building the model
emb_dim = 10
h_dim = 200

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, emb_dim), generator=g, requires_grad=True)
# Hidden layer
W1 = torch.randn((block_size * emb_dim, h_dim), generator=g, requires_grad=True)
b1 = torch.randn(h_dim, generator=g, requires_grad=True)
# Final layer
W2 = torch.randn((h_dim, vocab_size), generator=g, requires_grad=True)
b2 = torch.randn(vocab_size, generator=g, requires_grad=True)
# parameters
params = [C, W1, b1, W2, b2]
minibatch = 32

In [ ]:
epochs = 200000

# track stats
stepi = []
lossi = []

for epoch in range(epochs):
    # minibatch
    ix = torch.randint(0, Xtr.shape[0], (minibatch, ))
    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, block_size * emb_dim) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])
    stepi.append(epoch)
    lossi.append(loss.item())
    # backward pass
    for p in params:
        p.grad = None
    loss.backward()
    # updates
    # lr decay
    lr = 0.1 if epoch < 100000 else 0.01
    for p in params:
        p.data += -lr * p.grad

In [ ]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1, block_size * emb_dim) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
print(loss.item())

In [ ]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, block_size * emb_dim) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
print(loss.item())

In [ ]:
emb = C[Xtest]
h = torch.tanh(emb.view(-1, block_size * emb_dim) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytest)
print(loss.item())

In [ ]:
plt.plot(stepi, lossi)